In [1]:
!pip install click==7.1.1
!pip install MarkupSafe==1.1.1

!pip install flask==1.1.2

     |████████████████████████████████| 92kB 2.5MB/s 
  Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2


In [2]:
!pip install requests==2.21.0
!pip install certifi==2020.4.5.1
!pip install idna==2.8
!pip install click==7.1.1
!pip install MarkupSafe==1.1.1

!pip install flask-ngrok

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: idna 2.9
    Uninstalling idna-2.9:
      Successfully uninstalled idna-2.9
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


     |████████████████████████████████| 163kB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: certifi 2020.4.5.2
    Uninstalling certifi-2020.4.5.2:
      Successfully uninstalled certifi-2020.4.5.2


In [3]:
!pip install flask-cors

In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
from google.colab import drive 
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [6]:
# only required for importing modules

import sys
sys.path.append("/content/drive/My Drive/MinorProject2020_TextSummarization/PW20AMJ03/SourceCode/Objects")

In [7]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.utils import plot_model
import tensorflow
print(tensorflow.__version__)

1.15.2


In [8]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request 
from flask_cors import CORS, cross_origin

import numpy as np
import io
import json
import pickle
import re

from attention import AttentionLayer
from tensorflow.keras.models import load_model, Model
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping

path = "/content/drive/My Drive/MinorProject2020_TextSummarization/PW20AMJ03/SourceCode/Objects/"

Using TensorFlow backend.


In [9]:
def create_model():
    latent_dim = 300
    embedding_dim=100
    max_text_len = 30
    x_voc = 8440
    y_voc = 1989

    # tensorflow.keras.models.save_model(model, path + "model.h5")
    model = load_model(path+"model.h5", {"AttentionLayer":AttentionLayer})

    encoder_inputs = model.input[0]   # input_1
    encoder_outputs, state_h_enc, state_c_enc = model.layers[6].output
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = Model(encoder_inputs, [encoder_outputs] + encoder_states)

    decoder_inputs = model.input[1]   # input_2

    decoder_state_input_h = Input(shape=(latent_dim,), name="input_h")
    decoder_state_input_c = Input(shape=(latent_dim,), name="input_c")
    decoder_hidden_state_input = Input(shape=(max_text_len, latent_dim), name="input_hid")
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    dec_emb_layer = model.layers[5]
    dec_emb2 = dec_emb_layer(decoder_inputs)


    decoder_lstm = model.layers[7]
    decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)

    attn_layer = model.layers[8]
    attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])

    concat_layer = model.layers[9]
    decoder_inf_concat = concat_layer([decoder_outputs2, attn_out_inf])


    decoder_dense = model.layers[10]
    decoder_outputs2 = decoder_dense(decoder_inf_concat)
    decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input] + decoder_states_inputs, [decoder_outputs2] + [state_h2, state_c2])

    # print(encoder_model.summary())
    # print(decoder_model.summary())


    return encoder_model, decoder_model

def read_data():
    with open(path + "x_tokenizer.pkl", "rb") as f:
        x_tokenizer = pickle.load(f)

    return x_tokenizer 

def read_decode_data():

    with open(path + "var.pkl", "rb") as f:
        target_word_index, reversed_target_word_index, reverse_source_word_index = pickle.load(f)

    return target_word_index, reversed_target_word_index, reverse_source_word_index

def decode_sequence(input_seq):
    max_summary_len = 8

    target_word_index, reverse_target_word_index, reverse_source_word_index = read_decode_data()
    encoder_model, decoder_model = create_model()

    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

def pad(seq):
    if len(seq) < 30:
        pad = [0] * (30-len(seq))
        seq += pad
    else:
        seq = seq[:30]

    return seq

def get_summary(text):
    max_text_len = 30
    
    x_tokenizer = read_data()
    temp = x_tokenizer.texts_to_sequences([text])
    seq = temp[0]

    seq = pad(seq)
    #print(seq)
    #print("Review:",seq2text(seq))

    print(text)
    predicted_summary = decode_sequence(np.asarray(seq).reshape(1, max_text_len))
    print("Predicted summary:{}\n".format(predicted_summary))

    return predicted_summary

In [ ]:
app = Flask(__name__)

run_with_ngrok(app)

cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'


@app.route("/index", methods=["GET"])
@cross_origin()
def predict():

    if request.method == "GET":
        input_data = str(request.args.get("text"))
        predicted_summary = get_summary(input_data)

    return jsonify({"summary": predicted_summary, "success":"true"})

if __name__ == "__main__":
    # get_summary("this dog food is not good. i hate this a lot")
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    app.run()

* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://259478729ffb.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
I dislike the coffee. The flavor is bad.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


127.0.0.1 - - [23/Jun/2020 02:39:26] "GET /index?text=I%20dislike%20the%20coffee.%20The%20flavor%20is%20bad. HTTP/1.1" 200 -


Predicted summary: weak coffee

My cat likes the cookie very much. It is very yummy!


...

Predicted summary: cat loves it

worst mobile phone. dont buy it as it has got earpiece problem. while talking on phone echo voice is coming and phone vibrates while talking on phone. Flipkart also dont give you return while i was eligible for return just making fool to the customers.


127.0.0.1 - - [23/Jun/2020 03:31:44] "GET /index?text=worst%20mobile%20phone.%20dont%20buy%20it%20as%20it%20has%20got%20earpiece%20problem.%20while%20talking%20on%20phone%20echo%20voice%20is%20coming%20and%20phone%20vibrates%20while%20talking%20on%20phone.%20Flipkart%20also%20dont%20give%20you%20return%20while%20i%20was%20eligible%20for%20return%20just%20making%20fool%20to%20the%20customers. HTTP/1.1" 200 -


Predicted summary: awful

My cat likes the cookie very much. It is very yummy!


127.0.0.1 - - [23/Jun/2020 03:33:03] "GET /index?text=My%20cat%20likes%20the%20cookie%20very%20much.%20It%20is%20very%20yummy! HTTP/1.1" 200 -


Predicted summary: cat loves it

